<a href="https://colab.research.google.com/github/vitalfedorov/ML_Notes/blob/master/Metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd

In [0]:
y_true = [0, 1, 1, 0, 1, 1]
y_pred = [0, 1, 1, 0, 0, 1]

In [0]:
tp = 0 # true positive
tn = 0 # true negative
fp = 0 # false positive
fn = 0 # false negative

In [0]:
for i in range(len(y_true)):
  if y_true[i] == y_pred[i] == 1:
    tp += 1
  if y_true[i] == y_pred[i] == 0:
    tn += 1
  if y_true[i] == 1 and y_true[i] != y_pred[i]:
    fp += 1
  if y_true[i] == 0 and y_true[i] != y_pred[i]:
    fn += 1

In [0]:
print('True Positive (TP): {}'.format(tp))
print('True Negative (TN): {}'.format(tn))
print('False Positive (FP): {}'.format(fp))
print('False Negative (FN): {}'.format(fn))

True Positive (TP): 3
True Negative (TN): 2
False Positive (FP): 1
False Negative (FN): 0


### Recall (Sensitivity) or True Positive Rate (TPR)
\begin{equation*}
TRP = \frac{TP}{TP + FN}
\end{equation*}
With increase of FN, TRP will decline. If FN=0, TRP=1
### Precision or Positive Predictive Value (PPV)
\begin{equation*}
PPV = \frac{TP}{TP + FP}
\end{equation*}
With increse of FP, PPV will decline. If FP=0, PPV=1
### Accurace (ACC)

In [0]:
# Recall (Sensitivity) or True Positive Rate (TPR)
# TPR = TP / (TP + FN)
tpr = tp / (tp + fn)

# Precision or Positive Predictive Value (PPV)
# PPV = TP / (TP + FP) 
ppv = tp / (tp + fp)

In [0]:
print("Recall (TPR): {}".format(tpr))
print('Precision (PPV): {}'.format(ppv))

Recall (TPR): 1.0
Precision (PPV): 0.75
